In [6]:
import numpy as np
import torch 
import torchvision 
import os
from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch.utils.data import DataLoader
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import matplotlib.pyplot as plt
import transforms as T
from PIL import Image

In [9]:
split_folder = "training" if True else "validation"
img_folder = os.getcwd() + "/Food_data/training/" + split_folder +"images"
print(img_folder)

C:\Users\newto\OneDrive\Documents\Projects\New folder\Semantic_segmentation_of_food_images/Food_data/training/trainingimages


In [10]:
class FoodDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms, is_train=True):
        self.root = root
        self.transforms = transforms
        split_folder = "training" if is_train else "validation"
        
        self.img_folder = os.getcwd() + "/Food_data/training" + split_folder + "images"
        self.mask_folder = os.getcwd() + "/Food_data/training" + split_folder + "labels"
        
        # Filter Out Images with Empty masks
        self.imgs = []
        self.masks = []
        
        for mask_path in os.listdir(self.mask_folder):
            mask = Image.open(self.mask_folder + "/" + mask_path)
            mask = np.array(mask)[:,:,2]
            obj_ids = np.unique(mask)
            obj_ids = obj_ids[1:]
            if len(obj_ids) == 1:
                img_path = mask_path.replace('png','jpg')
                self.imgs.append(img_path)
                self.masks.append(mask_path)
                
    def __getitem__(self, idx):
        img_path = self.img_folder + "/" + self.imgs[idx]
        mask_path = self.mask_folder + "/" + self.masks[idx]
        
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)
        
        if img.size !=mask.size:
            img = img.resize(mask.size)
            
        mask = np.array(mask)[:,:,2]
        obj_ids = np.unique(mask)
        obj_ids = obj_ids[1:]
        
        masks = mask == obj_ids[:, None, None]
        
        num_objs = len(obj_ids)
        boxes = []
        
        for i in range(num_objs):
            pos = np.nonzero(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])
            
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.ones((num_objs), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        image_id = torch.tensor([idx])
        area = (boxes[:,3]-boxes[:,1]) * (boxes[:,2]-boxes[:,0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
            
        return img, target
    
    def __len__(self):
        return len(self.imgs)

In [11]:
def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.convertImageDtype(torch.float))
    
    if train:
        transforms.append((T.RandomHorizontalFlip(0.5)))
    return T.Compose(transforms)

In [12]:
def train_dataset():
    dataset = FoodDataset(root=root, transforms=get_transform(train=True), is_train=True)
    loader = DataLoader(dataset=dataset,
                        num_workers=0,
                        batch_size=1,
                        shuffle=False,
                        collate_fn=utils.collate_fn)
    return loader